# sPyTial Dataclass Builder Demo

This notebook shows how to build dataclass instances using sPyTial widgets with **automatic value tracking**!

## Simple Workflow:
1. Define your dataclass
2. Create a widget: `widget = spytial.dataclass_widget(MyClass)`
3. Use the visual interface to build data and export JSON
4. Get your dataclass automatically: `instance = widget.value`

The widget continuously observes the export directory and automatically updates `widget.value` when you export data from the interface. No manual JSON handling needed!

In [1]:
# Setup and imports
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import spytial
from dataclasses import dataclass, field, is_dataclass
from typing import List, Optional

import ipywidgets

## Step 1: Define Your Dataclass

In [2]:
@dataclass
@spytial.orientation(selector='name', directions=['above'])
class Person:
    name: str = ""
    age: int = 0
    email: str = ""
    

## Step 2: Create the Widget

In [3]:
# Create the widget with automatic value tracking
person_widget = spytial.dataclass_widget(Person)

person_widget

## Step 3: Get Your Built Dataclass

In [5]:
# Get the automatically tracked value
current_person = person_widget.value

if current_person:
    print("🎉 Success! Widget automatically tracked your data:")
    print(f"Name: {current_person.name}")
    print(f"Age: {current_person.age}")
    print(f"Email: {current_person.email}")
    print(f"Type: {type(current_person)}")
    print(f"Is Person instance: {isinstance(current_person, Person)}")
    
    # The instance is ready to use
    print(f"\n✅ Your dataclass instance 'current_person' is ready!")
    
else:
    print("⏳ No data captured yet.")
    print("Steps to get your dataclass:")
    print("1. Use the visual interface above")
    print("2. Build your person data")
    print("3. Click 'Export JSON' button")
    print("4. Run this cell again - widget.value will be automatically updated!")
    print("\n💡 The widget continuously watches for exports and updates the value.")

⏳ No data captured yet.
Steps to get your dataclass:
1. Use the visual interface above
2. Build your person data
3. Click 'Export JSON' button
4. Run this cell again - widget.value will be automatically updated!

💡 The widget continuously watches for exports and updates the value.


## That's It! Automatic Value Tracking

You've successfully built a dataclass instance with **automatic value tracking**! The workflow is:

1. `widget = spytial.dataclass_widget(MyClass)` - Create widget with file observer
2. Use the visual interface (iframe with full CDN access)
3. Click "Export JSON" when you've built your data
4. `instance = widget.value` - Get your dataclass automatically!

## Key Features:
- **Iframe rendering** - Full access to external CDNs and resources
- **Continuous observation** - Background thread watches for exported files
- **Automatic conversion** - JSON automatically becomes dataclass instances
- **Real-time updates** - `widget.value` always has the latest data
- **No manual steps** - No copying/pasting JSON required!

The widget observes the export directory and automatically converts any exported JSON to your dataclass type!

In [ ]:
# Quick test of widget creation
test_widget = spytial.dataclass_widget(Person)
print("Widget created successfully")

In [ ]:
# Real-time value checking demonstration
import time

print("🔍 Real-time value monitoring...")
print("Run this cell multiple times after exporting to see automatic updates!")

# Check widget value multiple times
for i in range(3):
    current_value = person_widget.value
    timestamp = time.strftime('%H:%M:%S')
    
    if current_value:
        print(f"⏰ {timestamp}: Found Person(name='{current_value.name}', age={current_value.age})")
    else:
        print(f"⏰ {timestamp}: No value yet - export from interface above")
    
    if i < 2:  # Don't sleep on last iteration
        time.sleep(1)

print(f"\n💡 widget.value = {person_widget.value}")
print("The value updates automatically when you export from the interface!")

In [ ]:
# Test the export functionality
widget_export_test = spytial.dataclass_widget(Person)
widget_export_test

In [ ]:
# Check if export worked
print("Current widget value:", widget_export_test.value)
if widget_export_test.value:
    print("Export successful!")
    print(f"Person: {widget_export_test.value}")
else:
    print("No value yet - try exporting from the interface above")